In [16]:
## Import modules
from tqdm import trange

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("GPU not found, using CPU")

In [2]:
## Load data
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=45),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

train_dataset = torchvision.datasets.CIFAR10(
    "data/cifar10", transform=transform, train=True, download=True
)
val_dataset = torchvision.datasets.CIFAR10(
    "data/cifar10", transform=transform, train=False, download=True
)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
## Load model
model = torchvision.models.resnet50(weights=True)
model.fc = nn.Sequential(
    nn.Linear(2048, 256), nn.ReLU(), nn.Linear(256, 10), nn.Sigmoid()
)
model = torch.nn.DataParallel(model)
model = model.to(device)

In [4]:
## Set hyperparameters
n_epochs = 10
batch_size = 64
learning_rate = 0.001

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
## Training run
iterator = trange(n_epochs, desc="Training", unit="steps")
for epoch in iterator:
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    iterator.set_postfix(train_loss="{:.5f}".format(loss.item()))

Training:   0%|          | 0/10 [00:12<?, ?steps/s]


KeyboardInterrupt: 

In [31]:
## Validation
n_correct = 0
for x, y in val_dataset:
    out = model(torch.unsqueeze(x, axis=0))
    pred = torch.argmax(out).item()
    if pred == y:
        n_correct += 1

print(f"Validation accuracy: {n_correct/len(val_dataset)}")

KeyboardInterrupt: 